In [ ]:
import requests
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()
ALPHAVANTAGE_API_KEY = os.getenv("ALPHAVANTAGE_API_KEY")

In [ ]:
import pandas as pd
import time

api_key = ALPHAVANTAGE_API_KEY

# List of currency pairs 
# List of popular currency pairs
currency_pairs = [
    ('EUR', 'USD'),  # Euro / US Dollar
    ('GBP', 'USD'),  # British Pound / US Dollar
    ('USD', 'JPY') # US Dollar / Japanese Yen
]


In [6]:
# Define a function to get historical forex data for a pair
def get_forex_data(from_symbol, to_symbol):
    url = f'https://www.alphavantage.co/query?function=FX_DAILY&from_symbol={from_symbol}&to_symbol={to_symbol}&apikey={api_key}&outputsize=full'
    response = requests.get(url)
    data = response.json()

    # Check if the response contains time series data
    if 'Time Series FX (Daily)' in data:
        df = pd.DataFrame.from_dict(data['Time Series FX (Daily)'], orient='index')
        df.columns = ['Open', 'High', 'Low', 'Close']
        df.index = pd.to_datetime(df.index)  # Convert index to datetime
        df['Currency Pair'] = f'{from_symbol}/{to_symbol}'  # Add currency pair column
        return df
    else:
        print(f"Error for {from_symbol}/{to_symbol}: {data.get('Error Message', 'Check API limit or parameters')}")
        return pd.DataFrame()


In [7]:
# Initialize an empty list to store dataframes
df_list = []

# Loop through the list of currency pairs and collect data
for from_symbol, to_symbol in currency_pairs:
    df = get_forex_data(from_symbol, to_symbol)
    if not df.empty:
        df_list.append(df)
    time.sleep(12)  # Sleep to respect API rate limit

# Combine all dataframes into a single dataframe
df_combined = pd.concat(df_list)

# Reset index and rename columns
df_combined.reset_index(inplace=True)
df_combined.rename(columns={'index': 'Date'}, inplace=True)

Error for EUR/USD: Check API limit or parameters
Error for GBP/USD: Check API limit or parameters
Error for USD/JPY: Check API limit or parameters


ValueError: No objects to concatenate

In [ ]:
# Display the first few rows of the combined dataframe
print(df_combined.head())
print(df_combined.tail())
print(df_combined.info())

In [ ]:
# Convert 'Open', 'High', 'Low', 'Close' columns to numeric
df_combined[['Open', 'High', 'Low', 'Close']] = df_combined[['Open', 'High', 'Low', 'Close']].apply(pd.to_numeric, errors='coerce')

# Verify the data types
print(df_combined.dtypes)

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

# Calculate daily returns using the 'Close' column
df_combined['Returns'] = df_combined.groupby('Currency Pair')['Close'].pct_change()

# Drop NaN values in the 'Returns' column
df_combined = df_combined.dropna(subset=['Returns'])

# Drop NaN values and select features for clustering
df_features = df_combined[['Returns']]

# Standardize the features
scaler = StandardScaler()
df_scaled = scaler.fit_transform(df_features)

# Apply K-Means Clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(df_scaled)

# Add cluster labels to the original dataframe (aligning index)
df_combined.loc[df_features.index, 'Cluster'] = clusters

# Display the first few rows of the updated dataframe
print(df_combined[['Currency Pair', 'Date', 'Returns', 'Cluster']].head())

In [ ]:
print(df_combined.head())
print(df_combined.tail())
print(df_combined.info())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting the clusters
plt.figure(figsize=(10, 6))
sns.scatterplot(data=df_combined, x='Date', y='Returns', hue='Cluster', palette='Set1')
plt.title('Clustering of Currency Pairs Based on Returns')
plt.xlabel('Date')
plt.ylabel('Returns')
plt.show()

In [ ]:
# Calculate average returns and volatility for each cluster
cluster_analysis = df_combined.groupby('Cluster')['Returns'].agg(['mean', 'std']).reset_index()
cluster_analysis.rename(columns={'mean': 'Avg_Return', 'std': 'Volatility'}, inplace=True)

# Display the cluster analysis
print(cluster_analysis)